In [14]:
len(df)

2670

In [10]:
test

,message_id,message,label
0,1717,lee to create new film superhero comic book ve...,3
1,471,german growth goes into reverse germany s econ...,1
2,278,mps issued with blackberry threat mps will be ...,4
3,1436,hamm bows out for us women s football legend m...,2
4,5,iraq advice claim sparks new row the tories sa...,4
...,...,...,...
440,272,china now top trader with japan china overtook...,1
441,1270,pc ownership to double by 2010 the number of...,0
442,1858,bollywood dvd fraudster is jailed a major dist...,3
443,1617,id theft surge hits us consumers almost a quar...,1


In [29]:
len(train_distribution[:round(len(train_distribution) * 0.80)])

1780

In [24]:
len(test), len(test_retained)

(445, 445)

In [19]:
len(y_train)

2136

In [27]:
len(test_distribution), len(test), len(test_retained)

(445, 445, 445)

In [32]:
# bbc_news

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import pickle
import csv
import numpy as np
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import sklearn
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

dataset = 'bbc_news' # folder and dataset name
# cols = ['gender', 'age', 'politics'] # outcome columns
cols = ['label_x'] # outcome columns
outcome = 'label'

logit = True

df = pd.read_csv("data/" + dataset + ".csv")

print(dataset)

# for model in ['GensimLDA', 'BERTopic', 'NMF', 'Mallet_LDA', 'CTM']:
for model in ['CTM']:
    print(model)
    scores = []
    scores_logit = []
    scores_rf = []
    scores_old = []

    rmses = []
    r2s = []
    maes = []

    for run in range(1, 2):

        print("Run: " + str(run))

        # loading in distributions that were saved during topic extraction
        test_distribution = pickle.load(open(dataset + '/' + model + '/run_' + str(run) + '/' + model + '_topic_distribution_test.pkl', 'rb'))
        test = pickle.load(open(dataset + '/' + model + '/run_' + str(run) + '/test.pkl', 'rb'))
        test_retained = pickle.load(open(dataset + '/' + model + '/run_' + str(run) + '/test_retained.pkl', 'rb'))

        if model == 'CTM':
            train_distribution = pickle.load(open(dataset + '/' + model + '/run_' + str(run) + '/' + model + '_topic_distribution_train.pkl', 'rb'))
            train_distribution = train_distribution[:round(len(train_distribution) * 0.80)]
        else:
            train_distribution = pickle.load(open(dataset + '/' + model + '/run_' + str(run) + '/' + model + '_topic_distribution_train.pkl', 'rb'))
        train = pickle.load(open(dataset + '/' + model + '/run_' + str(run) + '/train.pkl', 'rb'))
        train_retained = pickle.load(open(dataset + '/' + model + '/run_' + str(run) + '/train_retained.pkl', 'rb'))

        topics = []
        with open(dataset + '/' + model + '/run_' + str(run) + '/' + 'topics_100.txt', 'r') as f:
            reader = csv.reader(f)
            for row in reader:
                topic_list = [item.strip() for item in row if item.strip()]
                topics.append(topic_list)

        temp = pd.concat([train, test]).reset_index(drop=True) # concatenating train and test datasets
        distribution = np.concatenate([train_distribution, test_distribution]) # concatenating train and test distributions

        merged = pd.merge(temp, df, how='inner', left_on = 'message_id', right_on = 'Unnamed: 0')[['message_id', 'message_x'] + cols]
        merged.columns = ['message_id', 'message', 'label']

        X = distribution
        y = merged[outcome].reset_index(drop=True) # the outcome we care about

        # # 80-20 split --> didn't use train-test-split function since its already shuffled
        X_train = X[:round(0.80 * len(X))]
        X_test = X[round(0.80 * len(X)):]

        y_train = y[:round(0.80 * len(X))]
        y_test = y[round(0.80 * len(X)):]

        lr_model = LinearRegression().fit(X_train, y_train) 
        y_pred_test = lr_model.predict(X_test)

        pred_rounded = []

        for i in np.round(y_pred_test):
            min = y_test.min()
            max = y_test.max()
            if i >= min and i <= max:
                pred_rounded.append(i)
            elif i > max:
                pred_rounded.append(max)
            else:
                pred_rounded.append(min)

        score = sklearn.metrics.accuracy_score(pred_rounded, list(y_test))
        scores.append(score)

        rmse_test = mean_squared_error(y_true=y_test, y_pred=y_pred_test, squared=False)
        r2_test = r2_score(y_true=y_test, y_pred=y_pred_test)
        mae_test = mean_absolute_error(y_true=y_test, y_pred=y_pred_test)

        rmses.append(rmse_test)
        r2s.append(r2_test)
        maes.append(mae_test)

        logit_model = LogisticRegression().fit(X_train, y_train)
        pred_logit = logit_model.predict(X_test)
        score_logit = sklearn.metrics.accuracy_score(pred_logit, list(y_test))
        scores_logit.append(score_logit)

        # rf = RandomForestClassifier(random_state = 42).fit(X_train, y_train)
        # pred_rf = rf.predict(X_test)
        # score_rf = sklearn.metrics.accuracy_score(pred_rf, list(y_test))
        # scores_rf.append(score_rf)

        dummy_train = pd.get_dummies(train['label'], dtype=int)
        dummy_test = pd.get_dummies(test['label'], dtype=int)
        dummy_train.columns = ["label " + str(i) for i in range(len(dummy_train.columns))]
        dummy_test.columns = ["label " + str(i) for i in range(len(dummy_test.columns))]

        accuracies = []
        r2s_old = []
        for i, label_col in enumerate(list(dummy_train)):
            lr_model_old = LogisticRegression().fit(X_train, dummy_train[label_col])
            accuracies.append(lr_model_old.score(X_test, dummy_test[label_col]))
            r2s_old.append(r2_score(dummy_test[label_col], lr_model_old.predict(X_test)))
        
        accuracies = [np.mean(accuracies)] + accuracies + [np.mean(r2s_old)]
        scores_old.append(accuracies[0])
        
    print("Linear model score: " + str(np.mean(scores)))
    print("Linear model RMSE: " + str(np.mean(rmses)))
    print("Linear model R2: " + str(np.mean(r2s)))
    print("Linear model MAE: " + str(np.mean(maes)))
    print()
    print("Logistic model: " + str(np.mean(scores_logit)))
    print("Rf model: " + str(np.mean(scores_rf)))
    print("Old Logistic model: " + str(np.mean(scores_old)))
    print()

bbc_news
CTM
Run: 1
Linear model score: 0.7865168539325843
Linear model RMSE: 0.5660976545086227
Linear model R2: 0.8136061685141058
Linear model MAE: 0.35976824045254524

Logistic model: 0.946067415730337
Rf model: nan
Old Logistic model: 0.907865168539326

